# 載入需要資料/library

In [17]:
%env KERAS_BACKEND = tensorflow

env: KERAS_BACKEND=tensorflow


In [18]:
import keras
%matplotlib inline

import numpy as np
import matplotlib.pyplot as plt

from keras.models import Sequential, Model
from keras.layers import Dense, Activation, Lambda, concatenate
from keras.optimizers import SGD,Adam
from keras.layers import Input
from keras.datasets import mnist
from keras.utils import np_utils
from keras import metrics
from keras import backend as K

In [29]:
(x_train0 , y_train) , (x_test0 , y_test) = mnist.load_data()

In [30]:
x_train = x_train0.reshape(60000,-1)
x_test = x_test0.reshape(10000,-1)

x_train = x_train/255
x_test = x_test/255

# 模型建立

In [31]:
x = Input(shape=(784,))
enc_1 = Dense(150, activation = 'sigmoid')
enc_2 = Dense(2, activation='sigmoid')

enc_mean = Dense(2)
enc_log_var = Dense(2)
h = enc_2(enc_1(x))

dec_2 = Dense(150, activation = 'sigmoid')
dec_1 = Dense(784, activation = 'sigmoid')

x_hat = dec_1(dec_2(h))

autoencoder = Model(x, x_hat)
autoencoder.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         (None, 784)               0         
_________________________________________________________________
dense_13 (Dense)             (None, 150)               117750    
_________________________________________________________________
dense_14 (Dense)             (None, 2)                 302       
_________________________________________________________________
dense_17 (Dense)             (None, 150)               450       
_________________________________________________________________
dense_18 (Dense)             (None, 784)               118384    
Total params: 236,886
Trainable params: 236,886
Non-trainable params: 0
_________________________________________________________________


In [32]:
enc_x = enc_1(x)

z_mean = enc_mean(enc_x)
z_log_var = enc_log_var(enc_x)

In [33]:
def sampling(args):
    z_mean , z_log_var = args
    epsilon = K.random_normal(shape=(2,),mean=0. , stddev=1)
    return z_mean + K.exp(z_log_var / 2) * epsilon

In [34]:
z = Lambda(sampling, output_shape=(2,))([z_mean, z_log_var])
dec_x = dec_2(z)
x_hat = dec_1(dec_x)

VAE = Model(x, x_hat)
VAE.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            (None, 784)          0                                            
__________________________________________________________________________________________________
dense_13 (Dense)                (None, 150)          117750      input_3[0][0]                    
__________________________________________________________________________________________________
dense_15 (Dense)                (None, 2)            302         dense_13[1][0]                   
__________________________________________________________________________________________________
dense_16 (Dense)                (None, 2)            302         dense_13[1][0]                   
__________________________________________________________________________________________________
lambda_3 (

In [35]:
def vae_loss(x, x_recon):  
    recovery_loss = 784 * metrics.binary_crossentropy(x, x_recon)
    kl_loss = - 0.5 * K.sum(1 + z_log_var - K.square(z_mean) - K.exp(z_log_var), axis=-1)
    return recovery_loss + kl_loss

In [36]:
VAE.compile(loss=vae_loss, optimizer=Adam(),metrics=['accuracy'])
VAE.fit(x_train, y_train, batch_size=100, epochs=3)

Epoch 1/3
60000/60000 [==============================] - 49s 816us/step - loss: -38614.4889 - acc: 0.8139
Epoch 2/3
60000/60000 [==============================] - 37s 616us/step - loss: -43135.4824 - acc: 0.9949
Epoch 3/3
60000/60000 [==============================] - 40s 665us/step - loss: -43148.2821 - acc: 0.9951


In [37]:
VAE_Encoder = Model(x, z_mean)

VAE_Encoder.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         (None, 784)               0         
_________________________________________________________________
dense_13 (Dense)             (None, 150)               117750    
_________________________________________________________________
dense_15 (Dense)             (None, 2)                 302       
Total params: 118,052
Trainable params: 118,052
Non-trainable params: 0
_________________________________________________________________


In [38]:
h_input = Input(shape=(2,))
VAE_Decoder = Model(h_input, dec_1(dec_2(h_input)))

VAE_Decoder.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_4 (InputLayer)         (None, 2)                 0         
_________________________________________________________________
dense_17 (Dense)             (None, 150)               450       
_________________________________________________________________
dense_18 (Dense)             (None, 784)               118384    
Total params: 118,834
Trainable params: 118,834
Non-trainable params: 0
_________________________________________________________________


In [39]:
def choose_number(number):
    data_num = []
    for i in range (len(y_train0)):
        if y_train0[i] == number:
            data_num.append(i)
    idx = np.random.choice(data_num, 1)
    return idx 

In [47]:
def inBetween(idx_1, idx_2, t):
    data_0 = x_train0[idx_1]
    data_1 = x_train0[idx_2]
    data_t = (1-t)*x_train0[idx_1] + t*x_train0[idx_2]

    mu_0 = VAE_Encoder.predict(x_train[idx_1:idx_1+1]).squeeze()
    mu_1 = VAE_Encoder.predict(x_train[idx_2:idx_2+1]).squeeze()
    mu_t = (1-t)*mu_0 + t*mu_1

    plt.figure(figsize=(10, 10))
    ax1 = plt.subplot(2, 1, 2)
    ax1.scatter(mu_0[0], mu_0[1])
    ax1.scatter(mu_1[0], mu_1[1])
    ax1.scatter(mu_t[0], mu_t[1])

    ax2 = plt.subplot(2, 3, 1)
    ax2.imshow(data_0, cmap='Greys')

    ax3 = plt.subplot(2, 3, 2)
    ax3.imshow(data_t, cmap='Greys')  

    ax4 = plt.subplot(2, 3, 3)
    ax4.imshow(data_1, cmap='Greys')

In [48]:
from ipywidgets import interact, FloatSlider, interact_manual, IntSlider

In [49]:
interact(inBetween, idx_1=IntSlider(value = 1000, min = 1, max = 10000, step = 1),idx_2=IntSlider(value = 9000, min = 1, max = 10000, step = 1), t=FloatSlider(value=0.5, min=0, max=1.0,step=0.02,),)

interactive(children=(IntSlider(value=1000, description='idx_1', max=10000, min=1), IntSlider(value=9000, description='idx_2', max=10000, min=1), FloatSlider(value=0.5, description='t', max=1.0, step=0.02), Output()), _dom_classes=('widget-interact',))

<function __main__.inBetween>